# 19 

In [220]:
patterns = []
designs = []

with open("input.txt", "r") as file: 
    for id, line in enumerate(file):
        if id == 0: 
            patterns = [pattern.strip() for pattern in line.strip().split(",")]

        else: 
            if line.strip() == "": 
                continue

            designs.append(line.strip())

First, I implemented a silly solution (below) but then got another idea: look for an inteval "cover" of the string, then turn that into a graph and check if there is a path from start to finish.

This can be done "dynamically" while building the cover. 

In [221]:
import re 

def design_cover(design, patterns):
    """
    Gets all intervals that are covered by the patterns in the design. 
    """
    intervals = []
    for pattern in patterns: 
        regex = re.compile(pattern)
        for pattern_match in re.finditer(regex, design):
            intervals.append((pattern_match.start(), pattern_match.end()))
    return intervals

In [222]:
def is_covered(design, patterns):
    """ 
    Checks if the design is covered by the patterns.
    """

    # Get a list of intervals over the design 
    edges = design_cover(design, patterns)

    # Remove multiedges
    edges =  set(edges)

    # Create a graph from the intervals
    adjacency = dict()  # out edges

    for edge in edges: 
        if edge[0] not in adjacency:
            adjacency[edge[0]] = [edge[1]]
        else:
            adjacency[edge[0]].append(edge[1])
            
    # All of this is fast! 

    # DFS 
    stack = [0]
    discovered = set()

    while len(stack) > 0:
        current = stack.pop()

        if current == len(design): 
            return True

        if current in discovered: 
            continue
        else:
            discovered.add(current)

            if current in adjacency:
                stack.extend(adjacency[current])

    return False




In [ ]:
# This seems too low. 
result = 0

for design in designs: 

    #if not is_covered(design, patterns):
    #    print(design)
        
    result += is_covered(design, patterns)

print(result)

wuurugububruuwuggwuuwgbwwbrbwgrrrubwbuuwu
wwgwuurrurrbbgwuwwbgrbbrbruugwwgugbrbrgguuwrugwuguurbgwrru
wuugguggwrrgruwgbbrrgbwbruubgrbbrburgugurgwggwgggrrur
wuugrbbwuguwuwwuuruwrwbgurrbbwbwwrwuugbwgbuugbgwwrwwwgw
wuugrubwgggbugrbwgwgurgwuubuubrbrrbbruwbwrwgguwrrrwbwgbbw
wuugwgbwrrgugguwwgbwgbbwgbbgbgwuugbbuuwgrburuuurwbggwgrugu
wuugrbubwrbrgwubrggugwbrgbggbbgwurrwgwrwu
wuuurrguggwugbggggwbwbrrwurrbgwgwgbuwubgwubuuwbugwbwbwwwrguw
wuuurrbwrgwrguuwgrrgbuwggrbbwbuwbgugrwubwugwbrbg
wuugbbbwuuguuwgrrwrwgwwwwwrbwgwrbwguwbrbgugubr
wuuuurwgrwggwrruuuwubwubggrbrbwgbrwbgbwwurgbrwu
wuurggrwwbgwuwwrburuwrwrruruubgggbrugwwrwgbuwrbwgbur
wurwwgwuurbwwgrrbbubbbbgugwwurrggrurbwrbuurbbbb
wuugbgbbwuubbbrrgbgubbgrrrbrgugrbwguuwgburrwrguuwgrbg
wuurugrubwrrbbrbbwbubbgburrrbugwuwwrgrrrwwbwg
wuurrrwburwbbugubwwrgruwrrbgrwrrwwbwbbrruggbwuug
wuurbbrgubwrgurrugbwubbrbubruwbugrrbrwuwbbbbbbbwuwuw
wuuuugrgugugguwwrgurwurbbbrwbwwrgwuuwubgwru
wuugrwrwwrwbbgwgugugurrwbbwwwwugwwbuuurbwgwrbbggugwwwwbgur
wuurbbwggrbbgwrgurb

In [211]:
is_covered("bggr", patterns)

False

In [ ]:
# Silly solution first 
# for design in designs:
import re

def can_compose_design(design, patterns, steps = 10):

    regexes = {pattern:re.compile(pattern) for pattern in patterns}
    design_stubs = [design]


    # -----------------
    while len(design_stubs) > 0 and steps > 0: 
        print(design_stubs)
        design_stub = design_stubs.pop(0)
        # ------------- BFS 
        for pattern in patterns: 
            for pattern_match in re.finditer(regexes[pattern], design_stub):

                # Only store the interval that has been matched? 
                # For now, there is a an overcounting because the substrings 
                # are removed in a fixed order. 
                new_stub = design_stub[:pattern_match.start()] + design_stub[pattern_match.end():]
                if new_stub == "":
                    # print("Found a match: ", design_stub)
                    return True
                design_stubs.append(new_stub)

        steps -= 1

    return False

# Way too slow